Result from data version 464

## before you run, make sure you have enough RAM

#### Change the below value for whether or not you want to export data to ./output

In [1]:
output_result = False

below are some code rewritten from initial analysis

In [2]:
import re
import json
import pandas as pds
from os import listdir

In [3]:
data_files_location = {}
categories = {}

for f in listdir('./data'):
    if f.endswith('.json'):
        temp = json.load(open(f'./data/{f}'))
        for i in temp['items']:
            categories[i['id']] = i['snippet']['title']
    if f.endswith('.csv'):
        data_files_location[f[:2]] = f'./data/{f}'

if not categories:
    raise FileNotFoundError('can not locate any json category files')
    
if not data_files_location:
    raise FileNotFoundError('not CSV data files are located')

In [4]:
all_dfs = {}

for k, v in data_files_location.items():
    all_dfs[k] = pds.read_csv(v)

data processing code

In [5]:
def phrasing(s: str, k = 1):
    arr = re.findall(r"\w+'*\w+", s.lower())
    
    if len(arr) > k < 1:
        raise ValueError('Invalid array input or k value')
    ret = []
    for i in range(len(arr)+1-k):
        extract = arr[i:i+k]
        ret.append(' '.join(extract))
    return ret

def analyze_dataframe(label: str, df, max_phrase: int = 3):
    ret = {
        'trend category': {},
        'trend title phrase': {},
        'trend channels': {}
    }
    
    temp = {}
    channels_data = {}
        
    for i in df.itertuples():
        c = categories[str(i.categoryId)]
        
        try:
            temp[i.channelId]
        except KeyError:
            temp[i.channelId] = i.channelTitle
            
        try:
            channels_data[i.channelId] += 1
        except KeyError:
            channels_data[i.channelId] = 1
        
        try:
            ret['trend category'][c] += 1
        except KeyError:
            ret['trend category'][c] = 1
            
        result = phrasing(i.title, max_phrase)

        for r in result:
            try:
                ret['trend title phrase'][c]
            except KeyError:
                ret['trend title phrase'][c] = {}

            try:
                ret['trend title phrase'][c][r] += 1
            except KeyError:
                ret['trend title phrase'][c][r] = 1
                
    for k, v in channels_data.items():
        t = temp[k]
        try:
            ret['trend channels'][t]
        except KeyError:
            ret['trend channels'][t] = v
        else:
            ret['trend channels'][f'{t} ({k})'] = v
    
    return ret

In [6]:
output = {}

for k, v in all_dfs.items():
    output[k] = analyze_dataframe(k, v)


export result to ./output in *_analysis.json format

In [ ]:
if output_result:
    for k, v in output.items():
        with open(f'./output/{k}_analysis.json', 'w') as f:
            json.dump(v, f)

code for outputting result for display

In [8]:
def print_dict_sort(arr, re=False, limit=-1, indent: str = ''):
    count = 0
    # code from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    for i in sorted(arr.items(), key=lambda item: item[1], reverse=re):
        print(f'{indent}{i[1]}:\t{i[0]}')
        if limit != -1:
            count += 1
            if count >= limit:
                break

def print_output(out, re=True, limit=20):
    print('Numbers of Trend Video by Category:')
    print_dict_sort(out['trend category'], re, indent='\t')
    print('Channels and trend video count:')
    print_dict_sort(out['trend channels'], re, limit, indent='\t')
    
    print('\nTrend Video Title Separated By Category:')
    for k, v in out['trend title phrase'].items():
        print(f'\t{k}:')
        print_dict_sort(v, re, limit, '\t\t')
        if limit != -1:
            print('\t\t...')

### Analysis Data For Brazil

In [9]:
print_output(output['BR'])

Numbers of Trend Video by Category:
	24274:	Music
	24269:	Entertainment
	12990:	Sports
	12138:	People & Blogs
	6566:	Gaming
	4821:	Comedy
	1506:	News & Politics
	1486:	Education
	1423:	Science & Technology
	987:	Film & Animation
	741:	Howto & Style
	738:	Autos & Vehicles
	296:	Travel & Events
	127:	Pets & Animals
	31:	Nonprofits & Activism
Channels and trend video count:
	1057:	ge
	966:	Felipe Neto
	584:	FutParódias
	538:	De Placa
	512:	MasterChef Brasil
	486:	Free Fire - Brasil
	475:	WebTVBrasileira
	458:	Esporte Interativo
	449:	Invento na Hora
	439:	NOW UNITED
	438:	LOUD
	427:	AM3NlC
	404:	NBA Brasil
	387:	Dinah Moraes
	379:	UOL Esporte
	376:	FOX Sports Brasil
	374:	Carol Borba
	371:	Me poupe!
	369:	A Fazenda
	361:	NFA CHANNEL

Trend Video Title Separated By Category:
	People & Blogs:
		93:	grupos free fire
		72:	shark tank brasil
		69:	da maria alice
		68:	por um dia
		66:	lbff rodada grupos
		66:	rodada grupos free
		61:	stray kids skz
		58:	pela primeira vez
		58:	tour pelo meu
	

### Analysis Data For Canada

In [10]:
print_output(output['CA'])

Numbers of Trend Video by Category:
	19418:	Entertainment
	15283:	Gaming
	12952:	Music
	11006:	Sports
	8237:	People & Blogs
	5905:	Comedy
	3853:	Science & Technology
	3589:	Howto & Style
	3327:	News & Politics
	3008:	Film & Animation
	2502:	Education
	1996:	Autos & Vehicles
	623:	Travel & Events
	556:	Pets & Animals
	89:	Nonprofits & Activism
Channels and trend video count:
	486:	NBA
	379:	NHL
	333:	MrBeast Gaming
	323:	SSundee
	301:	MrBeast
	268:	DAZN Canada
	267:	Dude Perfect
	229:	The Game Theorists
	223:	Big Hit Labels
	220:	SMTOWN
	213:	ESPN
	212:	FORMULA 1
	212:	BLACKPINK
	204:	Veritasium
	203:	Fortnite
	202:	SpaceX
	202:	Tom Scott
	202:	NFL
	199:	First We Feast
	198:	The Try Guys

Trend Video Title Separated By Category:
	Music:
		2072:	official music video
		240:	official video ft
		221:	official lyric video
		156:	live from the
		125:	megan thee stallion
		116:	the kid laroi
		88:	lil durk official
		84:	tiny desk home
		84:	desk home concert
		83:	punjabi songs 2020
		80:	mac

### Analysis Data For Germany

In [11]:
print_output(output['DE'])

Numbers of Trend Video by Category:
	24083:	Entertainment
	17026:	Music
	11046:	Sports
	10177:	People & Blogs
	8550:	Gaming
	3915:	Autos & Vehicles
	3863:	Howto & Style
	3501:	Science & Technology
	3253:	News & Politics
	2559:	Comedy
	1861:	Education
	1427:	Film & Animation
	562:	Travel & Events
	434:	Pets & Animals
	87:	Nonprofits & Activism
Channels and trend video count:
	841:	Berlin - Tag & Nacht
	837:	JP Performance
	632:	Köln 50667
	590:	Sky Sport HD
	533:	tagesschau
	520:	Krass Schule - Die jungen Lehrer
	446:	Goal Deutschland
	426:	DAZN Bundesliga
	381:	Varion
	308:	DAZN UEFA Champions League
	306:	LUCA
	255:	Sportschau
	248:	BibisBeautyPalace
	240:	DAZN Länderspiele
	220:	Freshtorge
	210:	NFL
	209:	FORMULA 1
	199:	DFB
	198:	MrBeast
	188:	Team Harrison

Trend Video Title Separated By Category:
	Music:
		1142:	official music video
		344:	official video prod
		313:	video prod by
		223:	official lyric video
		143:	prod by jumpa
		116:	official video ft
		89:	prod by aside
		80:	of

### Analysis Data For France

In [12]:
print_output(output['FR'])

Numbers of Trend Video by Category:
	22418:	Entertainment
	19786:	Music
	9852:	Sports
	9147:	Comedy
	8836:	Gaming
	4668:	People & Blogs
	3733:	Howto & Style
	3240:	Science & Technology
	2685:	News & Politics
	2559:	Autos & Vehicles
	2515:	Education
	2079:	Film & Animation
	509:	Travel & Events
	279:	Pets & Animals
	85:	Nonprofits & Activism
Channels and trend video count:
	986:	beIN SPORTS France
	588:	HugoDécrypte
	546:	Michou
	524:	Ligue 1 Uber Eats
	509:	SQUEEZIE
	486:	RMC Sport
	470:	Amixem
	434:	Fuze III
	384:	CANAL+ Sport
	372:	JOYCA
	319:	Léna Situations
	316:	SQUEEZIE GAMING
	309:	Fédération Française de Football
	304:	Romain Dot Live
	303:	LeBouseuh
	300:	Benjamin Verrecchia
	300:	Pierre Croce
	297:	MichouOff
	286:	Mcfly et Carlito la chaîne secondaire
	283:	Inoxtag

Trend Video Title Separated By Category:
	Sports:
		253:	ligue des champions
		220:	equipe de france
		214:	paris saint germain
		206:	de france fff
		187:	ligue uber eats
		173:	france fff 2021
		168:	journée lig

### Analysis Data For Great Britain

In [13]:
print_output(output['GB'])

Numbers of Trend Video by Category:
	20910:	Entertainment
	17144:	Sports
	13341:	Gaming
	12322:	Music
	8407:	People & Blogs
	4637:	Comedy
	3419:	Science & Technology
	2654:	News & Politics
	2472:	Howto & Style
	2368:	Film & Animation
	1924:	Education
	1764:	Autos & Vehicles
	500:	Travel & Events
	458:	Pets & Animals
	75:	Nonprofits & Activism
Channels and trend video count:
	1124:	The United Stand
	1026:	Sky Sports Football
	762:	BT Sport
	496:	WWE
	394:	Jay's Virtual Pub Quiz
	379:	FORMULA 1
	333:	AFTV
	301:	MoreSidemen
	290:	MrBeast
	289:	SidemenReacts
	266:	ITV Sport
	265:	MrBeast Gaming
	252:	Sidemen
	250:	Dude Perfect
	245:	England
	243:	Sky Sports Boxing
	236:	Mark Goldbridge That's Football
	231:	carwow
	230:	BT Sport Boxing
	224:	SSundee

Trend Video Title Separated By Category:
	Entertainment:
		394:	virtual pub quiz
		376:	man utd transfer
		366:	utd transfer news
		356:	pub quiz live
		200:	quiz live thursday
		186:	the national lottery
		186:	national lottery lotto
		186:	l

### Analysis Data For India

In [14]:
print_output(output['IN'])

Numbers of Trend Video by Category:
	35172:	Entertainment
	14093:	Music
	10598:	People & Blogs
	6835:	Comedy
	3261:	Science & Technology
	3076:	Gaming
	3056:	Sports
	3018:	News & Politics
	2964:	Howto & Style
	1862:	Education
	1757:	Film & Animation
	515:	Autos & Vehicles
	513:	Travel & Events
	41:	Nonprofits & Activism
	21:	Pets & Animals
Channels and trend video count:
	1250:	Vijay Television
	1083:	Colors TV
	956:	ZEE5
	909:	SAB TV
	819:	Asianet
	694:	Crazy XYZ
	551:	Taarak Mehta Ka Ooltah Chashmah
	540:	T-Series
	512:	Colors Rishtey
	437:	MR. INDIAN HACKER
	406:	Flowers Comedy
	386:	Sun TV
	360:	Star Maa
	316:	ETV Jabardasth
	313:	SET India
	311:	Zee TV
	306:	Hungry Birds
	277:	Sourav Joshi Vlogs
	263:	Sony PAL
	256:	Rachit Rojha

Trend Video Title Separated By Category:
	Entertainment:
		1684:	mehta ka ooltah
		1677:	ka ooltah chashmah
		1670:	taarak mehta ka
		1277:	ooltah chashmah रक
		1277:	chashmah रक हत
		803:	before zee tv
		803:	zee tv hindi
		803:	tv hindi tv
		803:	hindi 

### Analysis Data For Japan

In [15]:
print_output(output['JP'])

Numbers of Trend Video by Category:
	30836:	Entertainment
	13699:	People & Blogs
	11350:	Music
	10129:	Gaming
	6898:	Sports
	4830:	Film & Animation
	4576:	Comedy
	4056:	Howto & Style
	1728:	Autos & Vehicles
	1498:	News & Politics
	1469:	Pets & Animals
	588:	Education
	488:	Science & Technology
	182:	Travel & Events
	62:	Nonprofits & Activism
Channels and trend video count:
	1324:	東海オンエア
	573:	Fischer's-フィッシャーズ-
	462:	コムドット
	438:	両学長 リベラルアーツ大学
	432:	もちまる日記
	390:	THE FIRST TAKE
	373:	はじめしゃちょー（hajime）
	361:	SixTONES
	359:	4月25日19時オープンのチャンネル
	356:	すとぷりちゃんねる
	353:	朝倉未来 Mikuru Asakura
	343:	48-フォーエイト
	337:	Snow Man
	315:	ヒカル（Hikaru）
	311:	P丸様。
	311:	ジャニーズJr.チャンネル
	299:	RIZIN FIGHTING FEDERATION
	293:	東海オンエアの控え室
	273:	HikakinTV
	272:	きまぐれクックKimagure Cook

Trend Video Title Separated By Category:
	Sports:
		166:	応援番組 虎バン abcテレビ公式チャンネル
		140:	阪神タイガース密着 応援番組 虎バン
		76:	yogibo presents rizin
		61:	uefa euro 2020tm
		61:	euro 2020tm サッカー欧州選手権
		61:	2020tm サッカー欧州選手権 wowow
		56:	番組 rizin confessions


### Analysis Data For South Korea

In [16]:
print_output(output['KR'])

Numbers of Trend Video by Category:
	31071:	Entertainment
	13107:	People & Blogs
	9387:	Music
	6927:	Sports
	5471:	Comedy
	5223:	News & Politics
	3666:	Howto & Style
	3123:	Film & Animation
	3045:	Gaming
	2120:	Education
	2041:	Pets & Animals
	1849:	Science & Technology
	1301:	Autos & Vehicles
	882:	Travel & Events
	141:	Nonprofits & Activism
Channels and trend video count:
	589:	짤툰
	584:	SPOTV
	418:	장삐쭈
	416:	파뿌리
	415:	BANGTANTV
	410:	총몇명
	408:	Mnet Official
	390:	Big Hit Labels
	390:	엠뚜루마뚜루 : MBC 공식 채널
	384:	런닝맨 - 스브스 공식 채널
	370:	워크맨-Workman
	369:	스포츠타임
	366:	채널 십오야
	353:	유 퀴즈 온 더 튜브
	343:	SMTOWN
	317:	문명특급 - MMTG
	310:	SBS NOW / SBS 공식 채널
	283:	달라스튜디오
	278:	웃소 Wootso
	277:	올리버쌤

Trend Video Title Separated By Category:
	Film & Animation:
		87:	컬투쇼 레전드 사연
		87:	레전드 사연 영상툰
		56:	among us animation
		52:	이해하면 무서운 이야기
		46:	dance 라이언 춘식
		46:	라이언 춘식 ep
		40:	영상툰 썰툰 일상툰
		40:	썰툰 일상툰 설렘썰
		40:	일상툰 설렘썰 공포썰
		40:	설렘썰 공포썰 고민썰
		40:	공포썰 고민썰 개그썰
		36:	squid game dalgona
		29:	이무이 시리즈 오싹툰
		29:

### Analysis Data For Mexico

In [17]:
print_output(output['MX'])

Numbers of Trend Video by Category:
	22805:	Music
	21498:	Entertainment
	11883:	People & Blogs
	11162:	Gaming
	8646:	Sports
	3994:	Comedy
	3610:	News & Politics
	3154:	Howto & Style
	1912:	Film & Animation
	1136:	Autos & Vehicles
	801:	Science & Technology
	786:	Travel & Events
	680:	Education
	120:	Nonprofits & Activism
	12:	Pets & Animals
Channels and trend video count:
	828:	TUDN México
	576:	Luisito Comunica
	555:	Cracks
	553:	Marca Claro
	452:	Más SKabeche
	448:	FOX Sports MX
	446:	Free Fire - LATAM
	436:	Kimberly Loaiza
	427:	Andrés Manuel López Obrador
	421:	BANGTANTV
	377:	Antrax
	364:	ESPN Deportes
	349:	Big Hit Labels
	320:	La Capital
	316:	JYP Entertainment
	315:	LosPolinesios
	299:	TheGrefg
	298:	LaLiga Santander
	285:	SMTOWN
	283:	Distrito Comedia

Trend Video Title Separated By Category:
	Music:
		905:	official music video
		235:	official lyric video
		197:	remix video oficial
		171:	el chombo presenta
		163:	official video ft
		161:	remix official video
		156:	los dos ca

### Analysis Data For Russia

In [18]:
print_output(output['RU'])

Numbers of Trend Video by Category:
	15663:	People & Blogs
	15038:	Entertainment
	11581:	News & Politics
	7752:	Music
	7484:	Sports
	6854:	Film & Animation
	6275:	Comedy
	5657:	Autos & Vehicles
	5255:	Gaming
	2766:	Howto & Style
	2700:	Science & Technology
	2100:	Nonprofits & Activism
	864:	Education
	697:	Travel & Events
	555:	Pets & Animals
Channels and trend video count:
	884:	Телеканал Дождь
	853:	Матч ТВ
	846:	Россия 24
	741:	Соловьёв LIVE
	732:	Первый канал
	707:	Анатолий Шарий
	674:	A4
	447:	Россия 1
	439:	НТВ
	400:	Lesha Maisak
	393:	Вести FM
	385:	Эхо Москвы
	335:	Навальный LIVE
	302:	Ozodivideo
	287:	Трум Трум
	276:	МУЗЫКА
	275:	Comedy Club
	271:	Редакция
	244:	PRO Английский футбол
	231:	Magic Five

Trend Video Title Separated By Category:
	Entertainment:
		284:	вечер владимиром соловьевым
		284:	владимиром соловьевым от
		145:	60 минут по
		145:	минут по горячим
		145:	по горячим следам
		126:	на 24 часа
		113:	от 123 go
		107:	горячим следам от
		84:	слепые прослушивания г

### Analysis Data For USA

In [19]:
print_output(output['US'])

Numbers of Trend Video by Category:
	18500:	Entertainment
	17123:	Music
	15645:	Gaming
	10165:	Sports
	7885:	People & Blogs
	5027:	Comedy
	3535:	Film & Animation
	3410:	News & Politics
	3381:	Science & Technology
	3057:	Howto & Style
	2022:	Education
	1659:	Autos & Vehicles
	500:	Pets & Animals
	394:	Travel & Events
	88:	Nonprofits & Activism
Channels and trend video count:
	580:	NBA
	577:	NFL
	492:	SSSniperWolf
	389:	MrBeast Gaming
	358:	SSundee
	330:	MrBeast
	319:	ESPN
	302:	Champions League on CBS Sports
	299:	Saturday Night Live
	287:	Dude Perfect
	277:	NBC Sports
	250:	The Game Theorists
	244:	Fortnite
	231:	SpaceX
	223:	beIN SPORTS USA
	214:	Veritasium
	214:	Nintendo
	213:	The Food Theorists
	211:	Apex Legends
	211:	Big Hit Labels

Trend Video Title Separated By Category:
	People & Blogs:
		173:	official music video
		136:	try not to
		136:	not to laugh
		124:	to laugh challenge
		120:	behind the scenes
		66:	the dixie d'amelio
		66:	dixie d'amelio show
		61:	here's what found
		